# Milestone 1

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Meet the Team
DSCI 525 - Group 12: 
Chen Lin, Edward Yukun Zhang, Jakob Thoms, Vikram Grewal

## Introduction

In this project, we are working with big data of daily rainfall in Australia on a large dataset. Here is [the link to our repo:](https://github.com/UBC-MDS/DSCI525_Group12). We first download the data from figshare to the local computer using the figshare API and then we combine data CSVs into a single CSV using pandas. Then we perform the EDA of summary statistics table of the min, max, mean, median and standard deviation of the all the columns. In R, we group by models to perform the summary statistics of EDA for the max rainfall, min rainfall, mean of rainfall and the median of rainfall across different models. 

### 3. Downloading the data

1. Download the data from figshare to your local computer using the figshare API:

In [2]:
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}

output_directory = "data/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]        
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
# Download the data in a zip file
files_to_dl = ["data.zip"] # Can add additional files to download
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

2. Extract the zip file programmatically:

In [5]:
# Unzip the data
with zipfile.ZipFile(os.path.join(output_directory, files_to_dl[0]), 'r') as f:
    f.extractall(output_directory)

### 4. Combining files

1. Combine data CSVs into a single CSV using pandas:
2. When combining the CSV files, add an extra column called "model" that identifies the model:

In [6]:
# Create list of files to combine
files = glob.glob('data/*.csv')
files.remove('data\\observed_daily_rainfall_SYD.csv')

In [7]:
# print one file to get an idea of how each individual file looks like 
df = pd.read_csv("data/BCC-ESM1_daily_rainfall_NSW.csv", index_col=0, parse_dates=True)
df

,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
time,,,,,
1889-01-01 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.286930e-24
1889-01-02 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.279780e-24
1889-01-03 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.499248e-17
1889-01-04 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.013407e-01
1889-01-05 12:00:00,-36.277805,-33.487232,142.03125,144.84375,4.578585e-01
...,...,...,...,...,...
2014-12-27 12:00:00,-30.696652,-27.906064,150.46875,153.28125,1.066399e-01
2014-12-28 12:00:00,-30.696652,-27.906064,150.46875,153.28125,1.622986e-09
2014-12-29 12:00:00,-30.696652,-27.906064,150.46875,153.28125,3.847284e-09


In [8]:
%%time
# Combine all CSVs into one dataframe
df = pd.concat((
  pd.read_csv(file, index_col=0, parse_dates=True).\
    assign(model=re.findall("data\\\\([^_]*)", file)[0]) # Add the 'model' column
  for file in files
))

df

CPU times: total: 36.8 s
Wall time: 2min 16s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.250000,-35.00000,140.625,142.500,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.250000,-35.00000,140.625,142.500,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.250000,-35.00000,140.625,142.500,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.250000,-35.00000,140.625,142.500,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.250000,-35.00000,140.625,142.500,1.047658e-02,ACCESS-CM2
...,...,...,...,...,...,...
2014-12-27 12:00:00,-30.157068,-29.21466,153.125,154.375,5.543748e-01,TaiESM1
2014-12-28 12:00:00,-30.157068,-29.21466,153.125,154.375,7.028577e+00,TaiESM1
2014-12-29 12:00:00,-30.157068,-29.21466,153.125,154.375,2.347570e-01,TaiESM1


In [9]:
%%time
# Save the combined dataframe to CSV
df.to_csv("data/combined_data.csv")

CPU times: total: 2min 49s
Wall time: 9min 7s


### 5. Load the combined CSV to memory and perform a simple EDA (Python)

1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA:
    - Changing dtype of your data
     - Load just columns that we want
     - Loading in chunks

In [17]:
# Changing the dtype of the data
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df.astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3498.20 MB
Memory usage with float32: 2248.84 MB


In [18]:
# Load just the columns that we want
print(f"Memory usage with all columns: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with only two columns: {df[['rain (mm/day)', 'model']].memory_usage().sum() / 1e6:.2f} MB")

Memory usage with all columns: 3498.20 MB
Memory usage with only two columns: 1499.23 MB


2. Compare run times on different machines within your team and summarize your observations: see the table below

In [ ]:
# %%time
# df = pd.read_csv("combined_data.csv")

In [20]:
%%time
# EDA with default dtype (float64) and using all columns
df.describe()

CPU times: total: 4.06 s
Wall time: 13 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [22]:
%%time
# EDA with changed dtype (float32) and using all columns
df.astype('float32', errors='ignore').describe()

CPU times: total: 5.02 s
Wall time: 9.05 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310463e+01,-3.197747e+01,1.469054e+02,1.482152e+02,1.901175e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [23]:
%%time
# EDA with default dtype (float64) and using only two columns
df[['rain (mm/day)', 'model']].describe()

CPU times: total: 250 ms
Wall time: 3.38 s


,rain (mm/day)
count,5.924854e+07
mean,1.901170e+00
std,5.585735e+00
min,-3.807373e-12
25%,3.838413e-06
50%,6.154947e-02
75%,1.020918e+00
max,4.329395e+02


### 6. Perform a simple EDA in R

In [24]:
import os
os.environ['R_HOME'] = '/Users/jakob/miniconda3/envs/525/Lib/R' # Set this to your R path 

In [25]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


1. Choose one of the methods listed below for transferring the dataframe (i.e., the entire dataset) from Python to R, and explain why you opted for this approach instead of the others:
    - Parquet file
     - Pandas exchange
    - Arrow exchange

In [26]:
filepathcsv = "data/combined_data.csv"
filepathparquet = "data/combined_data.parquet"
filepathparquetr = "data/combined_data_r.parquet"

In [27]:
# !pip install rpy2_arrow
import pyarrow.dataset as ds
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

We chose to use Arrow exchange because: 
1. it transfers the large data in a faster and more memory-efficient way. 
2. Moreover, it also preserves the data types and schema.

In [28]:
dataset = ds.dataset(filepathcsv, format="csv")
# Converting the `pyarrow dataset` to a `pyarrow table`
table = dataset.to_table()
# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

2. Once you have the dataframe in R, perform a simple EDA:

In [29]:
%%R -i r_table
start_time <- Sys.time()
suppressMessages(library(dplyr))

result <- r_table %>% 
    group_by(model) %>% 
    summarize(
        max_rain = max(`rain (mm/day)`),
        median_rain = median(`rain (mm/day)`),
        mean_rain = mean(`rain (mm/day)`),
        min_rain = min(`rain (mm/day)`)
    )

end_time <- Sys.time()
print(result %>% collect())
print(end_time - start_time)

# A tibble: 27 x 5
   model            max_rain median_rain mean_rain  min_rain
   <chr>               <dbl>       <dbl>     <dbl>     <dbl>
 1 ACCESS-CM2          433.       0.0998      1.79  0       
 2 ACCESS-ESM1-5       202.       0.136       2.22 -3.05e-18
 3 AWI-ESM-1-1-LR       89.5      0.0463      2.03  9.07e-14
 4 BCC-CSM2-MR         286.       0.0453      1.95  3.77e-24
 5 BCC-ESM1            164.       0.0127      1.81  9.26e-25
 6 CanESM5             174.       0.0878      1.89  0       
 7 CMCC-CM2-HR4        224.       0.271       2.28  0       
 8 CMCC-CM2-SR5        203.       0.219       2.38 -4.08e-13
 9 CMCC-ESM2           230.       0.204       2.27 -7.87e-14
10 EC-Earth3-Veg-LR    181.       0.0109      1.52 -1.12e-18
# i 17 more rows
# i Use `print(n = ...)` to see more rows
Time difference of 0.469944 secs


## Summary of Comparison Results 

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken for Q4| Time taken for Q5|
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|
|  Chen Lin   |     OSX 13.2     | 32GB|Apple M2 Max|   Yes   |   2min 58s | 28.4s |
|  Edward Yukun Zhang   |     OSX 13.2     | 16GB|Intel 10th-gen i-7 2.3Gz|   Yes   |   7min 18s | 30.4s |
| Vikram Grewal | Windows 10          | 16GB | Ryzen 5700u | Yes     | 7min 42s | 13.2s       |
|  Jakob Thoms   |   Windows 11   | 16GB| Intel 12th gen i7-1255U   1.70 GHz |   Yes   |   9min 7s | 13s |
